In [1]:
import sqlite3 as db
import pandas as pd
import featuretools as ft
import json

In [20]:
#
# medication: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False-
#
def read_csvs():
    datasets = [ 'admissiondrug', 'admissionDx', 'allergy', 'apacheApsVar', 'apachePatientResult', 'apachePredVar', 'carePlanCareProvider', 'carePlanEOL', 'carePlanGeneral',
                 'carePlanGoal', 'carePlanInfectiousDisease', 'customLab', 'diagnosis', 'hospital', 'infusiondrug', 'intakeOutput', 'lab',
                 'medication',
                 'microLab', 'note',
                 'nurseAssessment', 'nurseCare', 'nurseCharting', 'pastHistory', 'patient', 'physicalExam', 'respiratoryCare', 'respiratoryCharting', 'treatment', 'vitalAperiodic',
                 'vitalPeriodic']

    dfs = {}

    for ds_name in datasets:
        dfs[ds_name.lower()] = (pd.read_csv('../db/csv/' + ds_name + '.csv'), )
    
    return dfs

def read_query(q):
    conn = db.connect('../db/sqlite/eicu_v2_0_1.sqlite3')
    df = pd.read_sql_query(q, conn)
    conn.close()
    
    return df

def make_relationships(dfs):
    relationships = []
    pk_fk = json.loads( open('keys.json').read() )
    i = 0

    for ds_name in pk_fk:
        #if pk_fk[ds_name]['pk'] != False:
        #    dfs[ds_name][0].set_index(pk_fk[ds_name]['pk'])
        
        if pk_fk[ds_name]['fk'] != False and ds_name not in ('hospital', 'medication'):
            #print(ds_name, pk[ds_name]['fk'])
            fk_atr, target_table, target_atr = pk_fk[ds_name]['fk']
            
            #print((target_table, target_atr, ds_name, fk_atr))
            relationships.append((target_table, target_atr, ds_name, fk_atr))
            
    relationships.append(('hospital', 'hospitalid', 'patient', 'hospitalid'))

    return relationships

#------------------------------------------------------------------------------------------------------

dfs = read_csvs()
relationships = make_relationships(dfs)

#feature_matrix, feature_defs = ft.dfs(
#    dataframes=dfs,
#    relationships=relationships,
#    target_dataframe_name='patient',
#)

#dfs['patient'][0]

/tmp/ipykernel_21765/385061124.py:15: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs[ds_name.lower()] = (pd.read_csv('../db/csv/' + ds_name + '.csv'), )


In [ ]:
#read_query("""
#SELECT * FROM patient
#""")

In [ ]:
#read_query("""
#SELECT * FROM apacheApsVar INNER JOIN apachePatientResult ON (apacheApsVar.patientUnitStayID = apachePatientResult.patientUnitStayID)
#                           INNER JOIN apachePredVar ON (apacheApsVar.patientUnitStayID = apachePredVar.patientUnitStayID)
#""")

In [ ]:
#df[ df['unitdischargestatus'] == df['uniquepid'][0] ]
#len(patient_df[ patient_df['unitdischargestatus'] == 'Alive' ]) # 2392 alive, 126 dead, 2 NULL